In [ ]:
rom google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import random
import numpy as np
import os
import tensorflow as tf


#랜덤시드 고정

In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(42)

# 파이캐럿

- 설치 후 반드시 런타임 다시 시작을 해야함!+ 왜인지 모르게 파이캐럿에서 성능이 안나올 때도 있는데(session_id 설정줘도) 그때도 런타임 다시시작 후 돌려주세요.

In [ ]:
!pip install catboost
!pip install xgboost
!pip install pycaret[full]

!pip install markupsafe==2.0.1

from pycaret.utils import enable_colab
import jinja2

enable_colab()



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.0.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (31 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver 

## Import

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

## Data Load

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/[데이콘]신용카드 사기 거래 탐지 AI/data/train.csv') # Train
train_df = train_df.drop(["ID"],axis=1)
train_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,4.983721,-0.994972
1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,1.418291,-0.994972
2,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,-0.256131,-0.994960
3,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,1.249376,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,0.262698,-0.994901
4,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,-0.410430,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,0.994900,-0.994901


In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/[데이콘]신용카드 사기 거래 탐지 AI/data/val.csv') # Validation
test_df = test_df.drop(["ID"],axis=1)
test_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,Class
0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,-0.366846,...,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,-0.255991,-0.994878,0
1,0.962496,0.328461,-0.171479,2.109204,1.129566,1.696038,0.107712,0.521502,-1.191311,0.724396,...,0.402492,-0.048508,-1.371866,0.390814,0.199964,0.016371,-0.014605,0.168937,-0.994784,0
2,1.145524,0.575068,0.194008,2.598192,-0.092210,-1.044430,0.531588,-0.241888,-0.896287,0.757952,...,-0.119703,-0.076510,0.691320,0.633984,0.048741,-0.053192,0.016251,0.169496,-0.994502,0
3,0.927060,-0.323684,0.387585,0.544474,0.246787,1.650358,-0.427576,0.615371,0.226278,-0.225495,...,0.079359,0.096632,-0.992569,0.085096,0.377447,0.036096,-0.005960,0.331307,-0.994467,0
4,-3.005237,2.600138,1.483691,-2.418473,0.306326,-0.824575,2.065426,-1.829347,4.009259,6.051521,...,-0.181268,-0.163747,0.515821,0.136318,0.460054,-0.251259,-1.105751,-0.287012,-0.994373,0


# mcd 라벨링

In [ ]:
from sklearn.covariance import MinCovDet

In [ ]:
mcd_model = MinCovDet(support_fraction=1,random_state=42)
mcd_model.fit(train_df)

MinCovDet(random_state=42, support_fraction=1)

In [ ]:
import torch
# 상위 N개가 이상치라고 판단

def get_pred_label(model, x, k):
  prob = abs(mcd_model.mahalanobis(x))
  prob2 = torch.tensor(prob, dtype = torch.float)
  topk_indices = torch.topk(prob2, k = k, largest = True).indices

  pred = torch.zeros(len(x), dtype = torch.long)
  pred[topk_indices] = 1
  return pred.tolist(), prob2.tolist()


val_pred, val_prob = get_pred_label(mcd_model, train_df, 118)
train_df["Class"] = val_pred #라벨값 지정

# 파이캐럿 예측

In [ ]:
from pycaret.classification import *
reg1 = setup(train_df, target = 'Class',silent = True, session_id = 1974)

,Description,Value
0,session_id,1974
1,Target,Class
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(113842, 31)"
5,Missing Values,False
6,Numeric Features,30
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
# 2가지 모델 학습
lr_model = create_model("lr")
cb_model = create_model("catboost")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9999,1.0,1.0000,0.8889,0.9412,0.9411,0.9427
2,1.0000,1.0,1.0000,1.0000,1.0000,1.0000,1.0000
3,0.9999,1.0,1.0000,0.8889,0.9412,0.9411,0.9427
4,0.9999,1.0,0.8889,1.0000,0.9412,0.9411,0.9427
5,1.0000,1.0,1.0000,1.0000,1.0000,1.0000,1.0000
6,0.9997,1.0,0.7778,1.0000,0.8750,0.8749,0.8818
7,0.9999,1.0,0.8889,1.0000,0.9412,0.9411,0.9427
8,0.9997,1.0,0.8889,0.8889,0.8889,0.8888,0.8888


# TEST 불러오기




In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/[데이콘]신용카드 사기 거래 탐지 AI/data/test.csv') # test
test_df = test_df.drop(["ID"],axis=1)
test_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,1.783274,-0.994983
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.269825,-0.994983
2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0.670579,-0.994960
3,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,-0.099254,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,-0.237686,-0.994937
4,0.384978,0.616109,-0.874300,-0.094019,2.924584,3.317027,0.470455,0.538247,-0.558895,0.309755,...,0.049924,0.238422,0.009130,0.996710,-0.767315,-0.492208,0.042472,-0.054337,-0.167819,-0.994866


In [ ]:
prediction_test = test_df

def get_pred_label( model,x, k):

  prob =(model.predict_proba(test_df)[:,0])
  prob2 = torch.tensor(prob, dtype = torch.float)
  topk_indices = torch.topk(prob2, k = k, largest = False).indices

  pred = torch.zeros(len(x), dtype = torch.long)
  pred[topk_indices] = 1
  return pred.tolist(), prob2.tolist()

val_pred_lr, val_prob = get_pred_label( lr_model,prediction_test, 311)
val_pred_cat, val_prob = get_pred_label( cb_model,prediction_test, 305)


print("End!!")


End!!


In [ ]:
result = np.array(val_pred_cat) + np.array(val_pred_lr)
temp_result = np.where( result >=2,1,0) #최종 결과

## Submission

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/[데이콘]신용카드 사기 거래 탐지 AI/data/sample_submission.csv')
submit.head()

,ID,Class
0,AAAA0x1,1
1,AAAA0x2,1
2,AAAA0x5,1
3,AAAA0x7,1
4,AAAA0xc,1


In [ ]:
submit['Class'] = temp_result
submit.to_csv('./0804_submit_1.csv', index=False)